Карпеченко Д.М., dz23, v.0.1, 25.01.2023

In [1]:
import os
import sys

os.environ['PYSPARK_PYTHON'] = 'python'
os.environ['PYSPARK_DRIVER_PYTHON'] = 'jupyter'

In [10]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('winequality').getOrCreate()
df = spark.read.csv(r'D:\TeachMeSkills\DZ\dz23\winequality-red.csv', header = True, inferSchema = True, sep =';')
df.printSchema()

root
 |-- fixed acidity: double (nullable = true)
 |-- volatile acidity: double (nullable = true)
 |-- citric acid: double (nullable = true)
 |-- residual sugar: double (nullable = true)
 |-- chlorides: double (nullable = true)
 |-- free sulfur dioxide: double (nullable = true)
 |-- total sulfur dioxide: double (nullable = true)
 |-- density: double (nullable = true)
 |-- pH: double (nullable = true)
 |-- sulphates: double (nullable = true)
 |-- alcohol: double (nullable = true)
 |-- quality: integer (nullable = true)



In [11]:
df.show()

+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+
|fixed acidity|volatile acidity|citric acid|residual sugar|chlorides|free sulfur dioxide|total sulfur dioxide|density|  pH|sulphates|alcohol|quality|
+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+
|          7.4|             0.7|        0.0|           1.9|    0.076|               11.0|                34.0| 0.9978|3.51|     0.56|    9.4|      5|
|          7.8|            0.88|        0.0|           2.6|    0.098|               25.0|                67.0| 0.9968| 3.2|     0.68|    9.8|      5|
|          7.8|            0.76|       0.04|           2.3|    0.092|               15.0|                54.0|  0.997|3.26|     0.65|    9.8|      5|
|         11.2|            0.28|       0.56|           1.9|    0.075|               17.0|           

In [12]:
import pandas as pd
pd.DataFrame(df.take(5), columns=df.columns)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [13]:
df.count()

1599

In [14]:
df.dropna()

DataFrame[fixed acidity: double, volatile acidity: double, citric acid: double, residual sugar: double, chlorides: double, free sulfur dioxide: double, total sulfur dioxide: double, density: double, pH: double, sulphates: double, alcohol: double, quality: int]

In [15]:
df.count()

1599

In [16]:
df.dropDuplicates()

DataFrame[fixed acidity: double, volatile acidity: double, citric acid: double, residual sugar: double, chlorides: double, free sulfur dioxide: double, total sulfur dioxide: double, density: double, pH: double, sulphates: double, alcohol: double, quality: int]

In [17]:
df.count()

1599

In [18]:
from pyspark.sql.functions import col,isnan, when, count
df.select([count(when(isnan(a) | col(a).isNull(), a)).alias(a) for a in df.columns]).show()

+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+---+---------+-------+-------+
|fixed acidity|volatile acidity|citric acid|residual sugar|chlorides|free sulfur dioxide|total sulfur dioxide|density| pH|sulphates|alcohol|quality|
+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+---+---------+-------+-------+
|            0|               0|          0|             0|        0|                  0|                   0|      0|  0|        0|      0|      0|
+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+---+---------+-------+-------+



In [27]:
numcol=df.columns[:11]

In [28]:
from pyspark.ml.feature import StringIndexer, VectorAssembler
assembler = VectorAssembler(inputCols=numcol, outputCol="features")
df = assembler.transform(df)
df.show()

+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+--------------------+
|fixed acidity|volatile acidity|citric acid|residual sugar|chlorides|free sulfur dioxide|total sulfur dioxide|density|  pH|sulphates|alcohol|quality|            features|
+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+--------------------+
|          7.4|             0.7|        0.0|           1.9|    0.076|               11.0|                34.0| 0.9978|3.51|     0.56|    9.4|      5|[7.4,0.7,0.0,1.9,...|
|          7.8|            0.88|        0.0|           2.6|    0.098|               25.0|                67.0| 0.9968| 3.2|     0.68|    9.8|      5|[7.8,0.88,0.0,2.6...|
|          7.8|            0.76|       0.04|           2.3|    0.092|               15.0|                54.0|  0.997|3.26|     0.65|    9.8|    

In [30]:
pd.DataFrame(df.take(5), columns=df.columns)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,features
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,"[7.4, 0.7, 0.0, 1.9, 0.076, 11.0, 34.0, 0.9978..."
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,"[7.8, 0.88, 0.0, 2.6, 0.098, 25.0, 67.0, 0.996..."
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5,"[7.8, 0.76, 0.04, 2.3, 0.092, 15.0, 54.0, 0.99..."
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6,"[11.2, 0.28, 0.56, 1.9, 0.075, 17.0, 60.0, 0.9..."
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,"[7.4, 0.7, 0.0, 1.9, 0.076, 11.0, 34.0, 0.9978..."


In [31]:
train, test = df.randomSplit([0.7, 0.3], seed = 2023)
print("Training Dataset Count: " + str(train.count()))
print("Test Dataset Count: " + str(test.count()))

Training Dataset Count: 1136
Test Dataset Count: 463


In [32]:
from pyspark.ml.classification import RandomForestClassifier
rf = RandomForestClassifier(featuresCol = 'features', labelCol = 'quality')
rfModel = rf.fit(train)
predictions = rfModel.transform(test)
predictions.select('quality', 'rawPrediction', 'prediction', 'probability').show(25)

+-------+--------------------+----------+--------------------+
|quality|       rawPrediction|prediction|         probability|
+-------+--------------------+----------+--------------------+
|      8|[0.0,0.0,0.0,0.0,...|       6.0|[0.0,0.0,0.0,0.0,...|
|      7|[0.0,0.0,0.0,0.0,...|       6.0|[0.0,0.0,0.0,0.0,...|
|      7|[0.0,0.0,0.0,0.0,...|       6.0|[0.0,0.0,0.0,0.0,...|
|      6|[0.0,0.0,0.0,0.0,...|       6.0|[0.0,0.0,0.0,0.0,...|
|      7|[0.0,0.0,0.0,0.0,...|       6.0|[0.0,0.0,0.0,0.0,...|
|      7|[0.0,0.0,0.0,0.0,...|       6.0|[0.0,0.0,0.0,0.0,...|
|      7|[0.0,0.0,0.0,0.0,...|       6.0|[0.0,0.0,0.0,0.0,...|
|      8|[0.0,0.0,0.0,0.0,...|       6.0|[0.0,0.0,0.0,0.0,...|
|      5|[0.0,0.0,0.0,0.0,...|       6.0|[0.0,0.0,0.0,0.0,...|
|      5|[0.0,0.0,0.0,0.03...|       6.0|[0.0,0.0,0.0,0.00...|
|      5|[0.0,0.0,0.0,0.29...|       6.0|[0.0,0.0,0.0,0.01...|
|      6|[0.0,0.0,0.0,0.0,...|       6.0|[0.0,0.0,0.0,0.0,...|
|      5|[0.0,0.0,0.0,0.05...|       6.0|[0.0,0.0,0.0,0

In [33]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(labelCol="quality", predictionCol="prediction")
accuracy = evaluator.evaluate(predictions)
print("Accuracy = %s" % (accuracy))
print("Test Error = %s" % (1.0 - accuracy))

Accuracy = 0.5916914560731927
Test Error = 0.40830854392680727


In [34]:
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.sql.types import FloatType
import pyspark.sql.functions as F

preds_and_labels = predictions.select(['prediction','quality']).withColumn('quality', F.col('quality').cast(FloatType())).orderBy('prediction')
preds_and_labels = preds_and_labels.select(['prediction','quality'])
metrics = MulticlassMetrics(preds_and_labels.rdd.map(tuple))
print(metrics.confusionMatrix().toArray())

C:\Users\HP\anaconda3\envs\My\lib\site-packages\pyspark\sql\context.py:157: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


[[  0.   0.   4.   0.   0.   0.]
 [  0.   0.  10.   6.   0.   0.]
 [  0.   0. 153.  34.   0.   0.]
 [  0.   0.  78. 118.   2.   0.]
 [  0.   0.   5.  34.  16.   0.]
 [  0.   0.   0.   3.   0.   0.]]


['fixed acidity',
 'volatile acidity',
 'citric acid',
 'residual sugar',
 'chlorides',
 'free sulfur dioxide',
 'total sulfur dioxide',
 'density',
 'pH',
 'sulphates',
 'alcohol',
 'quality']

In [36]:
df2=df

In [38]:
df2.drop('features')

DataFrame[fixed acidity: double, volatile acidity: double, citric acid: double, residual sugar: double, chlorides: double, free sulfur dioxide: double, total sulfur dioxide: double, density: double, pH: double, sulphates: double, alcohol: double, quality: int]

Маленькая точность предсказанных значений из-за несбалансированных классов

In [39]:
df2.groupBy("quality").count().show() 

+-------+-----+
|quality|count|
+-------+-----+
|      6|  638|
|      3|   10|
|      5|  681|
|      4|   53|
|      8|   18|
|      7|  199|
+-------+-----+

